#  1.Initial Setup, Kaggle Connection, and Dat Download

In [ ]:


from google.colab import drive
import os
import shutil
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TARGET_SIZE = (96, 96)
INPUT_SHAPE = (TARGET_SIZE[0], TARGET_SIZE[1], 3)
BATCH_SIZE = 64
EPOCHS_PHASE_1 = 20
EPOCHS_PHASE_2 = 10
LEARNING_RATE_PHASE_1 = 0.001
LEARNING_RATE_PHASE_2 = 0.0001

KAGGLE_CREDENTIALS_PATH = "/content/drive/MyDrive/Kaggle/kaggle.json"
KAGGLE_DATASET_ID = "gti-upm/leapgestrecog"

DOWNLOAD_DIR = "/content/leap_gesture_data"

DATA_ROOT_PATH_RAW_BASE = os.path.join(DOWNLOAD_DIR, 'leapgestrecog')
FIXED_ROOT_PATH = '/content/leap_gesture_data/fixed_data'
MODEL_SAVE_PATH = "/content/drive/MyDrive/Kaggle_Gesture_Dataset/best_resnet50_leap_model.h5"

if os.path.exists(DOWNLOAD_DIR):
    shutil.rmtree(DOWNLOAD_DIR)
    print(f"Removed previous data directory: {DOWNLOAD_DIR}")
if os.path.exists(FIXED_ROOT_PATH):
    shutil.rmtree(FIXED_ROOT_PATH)
    print(f"Removed previous fixed directory: {FIXED_ROOT_PATH}")

drive.mount('/content/drive')
print("\n--- Setting up Kaggle API and downloading data ---")

!pip install -q kaggle

if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")

try:
    shutil.copy(KAGGLE_CREDENTIALS_PATH, "/root/.kaggle/kaggle.json")
    os.chmod("/root/.kaggle/kaggle.json", 0o600)
    print("Kaggle API key configured successfully.")
except FileNotFoundError:
    print(f"FATAL ERROR: Kaggle key not found at {KAGGLE_CREDENTIALS_PATH}. Please check the path.")
    exit()

os.makedirs(DOWNLOAD_DIR)
print("Downloading dataset...")
!kaggle datasets download -d {KAGGLE_DATASET_ID} -p {DOWNLOAD_DIR} --unzip
print(f"Dataset downloaded and unzipped to {DOWNLOAD_DIR}")

Removed previous data directory: /content/leap_gesture_data
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- Setting up Kaggle API and downloading data ---
FATAL ERROR: Kaggle key not found at /content/drive/MyDrive/Kaggle/kaggle.json. Please check the path.
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
          

# 2. Defining the Correct Path and Comprehensive Restructuring


In [ ]:


print("\n--- Starting robust data reorganization to ensure all 20,000 files are moved ---")


DATA_ROOT_PATH_FINAL = os.path.join(DATA_ROOT_PATH_RAW_BASE, 'leapGestRecog')

if not os.path.exists(DATA_ROOT_PATH_FINAL):
    print("CRITICAL: Final raw data path not found. Please check manually!")

    DATA_ROOT_PATH_FINAL = DATA_ROOT_PATH_RAW_BASE
else:
    print(f"Using confirmed raw data path: {DATA_ROOT_PATH_FINAL}")

if not os.path.exists(FIXED_ROOT_PATH):
    os.makedirs(FIXED_ROOT_PATH)

files_moved_count = 0

for subject_folder in os.listdir(DATA_ROOT_PATH_FINAL):
    subject_path = os.path.join(DATA_ROOT_PATH_FINAL, subject_folder)

    if os.path.isdir(subject_path) and subject_folder.isdigit():

        for gesture_folder in os.listdir(subject_path):
            source_gesture_path = os.path.join(subject_path, gesture_folder)

            if os.path.isdir(source_gesture_path):
                destination_path = os.path.join(FIXED_ROOT_PATH, gesture_folder)

                if not os.path.exists(destination_path):
                    os.makedirs(destination_path)


                    if filename.endswith('.png'):
                        shutil.move(os.path.join(source_gesture_path, filename), destination_path)
                        files_moved_count += 1

print(f"Data reorganization complete. Total files moved: {files_moved_count}")

print(f"\n--- Final verification of file count ---")
total_files = !find {FIXED_ROOT_PATH} -type f -name "*.png" | wc -l
try:
    total_files_count = int(total_files[0].strip())
    print(f"TOTAL FINAL PNG FILES FOUND: {total_files_count}")

    if total_files_count < 20000:
        print("CRITICAL WARNING: Files are missing. Expected 20,000. Proceed with caution.")
    else:
        print("SUCCESS: File count is correct (20,000 files expected).")

except:
    print("Could not count files automatically.")


--- Starting robust data reorganization to ensure all 20,000 files are moved ---


NameError: name 'os' is not defined

#  3. Data Generator Setup


In [ ]:


print("\n--- Data Preprocessing and Generator Setup ---")


def grayscale_to_rgb(x):

    return tf.concat([x, x, x], axis=-1)

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range = 0.15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1
)


train_generator = datagen.flow_from_directory(
    FIXED_ROOT_PATH,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    FIXED_ROOT_PATH,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

NUM_CLASSES = train_generator.num_classes
print(f"Number of Gesture Classes detected: {NUM_CLASSES}")


--- Data Preprocessing and Generator Setup ---


NameError: name 'ImageDataGenerator' is not defined

#  4. Build ResNet Model


In [ ]:


print("\n--- Building ResNet50 Model (Transfer Learning Setup) ---")


def build_resnet_transfer_model_for_leap(num_classes):
    input_tensor = Input(shape=(TARGET_SIZE[0], TARGET_SIZE[1], 1))
    rgb_like_output = Lambda(grayscale_to_rgb,
                             output_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3))(input_tensor)


    resnet_base = ResNet50(weights='imagenet',
                           include_top=False,
                           input_tensor=rgb_like_output,
                           name='resnet_base_model')

    resnet_base.trainable = False


    x = resnet_base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax', name='predictions')(x)

    return Model(inputs=input_tensor, outputs=output)

resnet_transfer_model = build_resnet_transfer_model_for_leap(NUM_CLASSES)
print("Model built successfully with frozen ResNet base.")


--- Building ResNet50 Model (Transfer Learning Setup) ---


NameError: name 'NUM_CLASSES' is not defined

# 5. Initial Training (Phase 1: Classification Head Training)


In [ ]:

callbacks_phase1 = [
    EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True, verbose=1),

    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001, verbose=1),

    ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, mode='max')
]

resnet_transfer_model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_PHASE_1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Starting Phase 1: Training Classification Head ---")
history_phase1 = resnet_transfer_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS_PHASE_1,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=callbacks_phase1
)

# 6.  Performance Optimization (Phase 2: Fine-Tuning)


In [ ]:

def grayscale_to_rgb(x):
    return tf.concat([x, x, x], axis=-1)

print("\nLoading best model from Phase 1...")
resnet_transfer_model = tf.keras.models.load_model(
    MODEL_SAVE_PATH,
    custom_objects={'grayscale_to_rgb': grayscale_to_rgb}
)


for layer in resnet_transfer_model.layers:
    layer.trainable = True


RESNET_START_INDEX = 2
fine_tune_at_resnet_layers = 140
freeze_until_index = RESNET_START_INDEX + fine_tune_at_resnet_layers

frozen_count = 0
for i in range(RESNET_START_INDEX, freeze_until_index):
    if i < len(resnet_transfer_model.layers):
        layer = resnet_transfer_model.layers[i]
        layer.trainable = False
        frozen_count += 1

print(f"Total layers unfrozen for Fine-Tuning (ResNet upper layers + Classification Head): {len(resnet_transfer_model.layers) - frozen_count}")
print(f"Frozen {frozen_count} ResNet layers (indices 2 to {freeze_until_index - 1}).")

total_epochs = EPOCHS_PHASE_1 + EPOCHS_PHASE_2

callbacks_phase2 = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001, verbose=1),
    ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, mode='max')
]

resnet_transfer_model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_PHASE_2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"\n--- Starting Phase 2: Fine-Tuning (Training upper {len(resnet_transfer_model.layers) - frozen_count} layers) ---")

history_fine = resnet_transfer_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=total_epochs,
    initial_epoch=history_phase1.epoch[-1],
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=callbacks_phase2
)

# 7 Final Evaluation and Plotting Charts


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
import itertools
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Import the library again

def grayscale_to_rgb(x):
    return tf.concat([x, x, x], axis=-1)

print("\n--- Final Model Evaluation and Visualization (Applying Shuffle=False Fix) ---")

final_model = tf.keras.models.load_model(
    MODEL_SAVE_PATH,
    custom_objects={'grayscale_to_rgb': grayscale_to_rgb}
)


eval_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

eval_generator = eval_datagen.flow_from_directory(
    FIXED_ROOT_PATH,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


history_combined = {}

for key in history_phase1.history.keys():
    history_combined[key] = (history_phase1.history[key] + history_fine.history[key])

plt.figure(figsize=(12, 5))
epochs_range = range(len(history_combined['accuracy']))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, history_combined['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history_combined['val_accuracy'], label='Validation Accuracy')

plt.axvline(x=EPOCHS_PHASE_1, color='r', linestyle='--', label='Phase 2 Start')
plt.title('Accuracy History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, history_combined['loss'], label='Training Loss')
plt.plot(epochs_range, history_combined['val_loss'], label='Validation Loss')
plt.axvline(x=EPOCHS_PHASE_1, color='r', linestyle='--', label='Phase 2 Start')
plt.title('Loss History')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('accuracy_loss_history.png')
plt.show()

eval_generator.reset()
STEP_SIZE_VALID_FULL = eval_generator.samples // eval_generator.batch_size + (eval_generator.samples % eval_generator.batch_size != 0)

print("\nCalculating predictions using non-shuffling generator...")
Y_pred = final_model.predict(eval_generator, steps=STEP_SIZE_VALID_FULL, verbose=1)
y_pred_classes = np.argmax(Y_pred, axis=1)

y_true = eval_generator.classes[0:len(y_pred_classes)]
class_names = list(eval_generator.class_indices.keys())

print("\n" + "="*80)
print("Classification Report (Precision, Recall, F1-Score) after Fine-Tuning")
print("================================================================================\n")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = '.2f'
    else:
        fmt = 'd'

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, fmt.format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cm, classes=class_names, title='Confusion Matrix for ResNet50 Fine-Tuning')
plt.savefig('confusion_matrix.png')
plt.show()

loss, accuracy = final_model.evaluate(eval_generator, steps=eval_generator.samples // BATCH_SIZE, verbose=1)

print(f"\n--- Final Confirmed Performance Summary ---")
print(f"Final Validation Accuracy (Confirmed): {accuracy * 100:.4f}%")
print(f"Final Validation Loss (Confirmed): {loss:.4f}")
print(f"Best model saved to: {MODEL_SAVE_PATH}")